# Определение уровня потребительской лояльности (NPS)

**Заказчик исследования** — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ.  
Перед компанией стоит *задача определить текущий уровень потребительской лояльности, или NPS* (от англ. Net Promoter Score),  
среди клиентов из России.  

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».  

Компания провела опрос и попросила подготовить дашборд с его итогами.  
Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite. 

## Вопросы исследования

- Как распределены участники опроса по возрасту и полу? Каких пользователей больше: новых или старых? Пользователи из каких городов активнее участвовали в опросе?
- Какие группы пользователей наиболее лояльны к сервису? Какие менее?
- Какой общий NPS среди всех опрошенных?
- Как можно описать клиентов, которые относятся к группе cторонников (англ. promoters)?

## Описание данных

База состоит из 5 следующих таблиц.

### Таблица **user**  
Содержит основную информацию о клиентах.  

*Состав*: 
- **user_id**	- Идентификатор клиента, первичный ключ таблицы
- **lt_day**	- Количество дней «жизни» клиента
- **age** - Возраст клиента в годах
- **gender_segment**	- Пол клиента (1 – женщина, 0 – мужчина)
- **os_name** - Тип операционной системы
- **cpe_type_name**	- Тип устройства
- **location_id**	- Идентификатор домашнего региона клиента, внешний ключ, отсылающий к таблице location
- **age_gr_id**	- Идентификатор возрастного сегмента клиента, внешний ключ, отсылающий к таблице age_segment
- **tr_gr_id**	- Идентификатор сегмента клиента по объёму потребляемого трафика в месяц, внешний ключ, отсылающий к таблице traffic_segment
- **lt_gr_id**	- Идентификатор сегмента клиента по количеству месяцев «жизни», внешний ключ, отсылающий к таблице lifetime_segment
- **nps_score**	- Оценка клиента в NPS-опросе (от 1 до 10)


### Таблица **location**  
Справочник территорий, в которых телеком-компания оказывает услуги.  

*Состав*: 
- **location_id**	- Идентификатор записи, первичный ключ
- **country**	- Страна
- **city**	- Город  

### Таблица **age_segment**  
Данные о возрастных сегментах клиентов.  

*Состав*: 
- **age_gr_id**	- Идентификатор сегмента, первичный ключ
- **bucket_min**	- Минимальная граница сегмента
- **bucket_max**	- Максимальная граница сегмента
- **title**	- Название сегмента  

### Таблица **traffic_segment**  
Данные о выделяемых сегментах по объёму потребляемого трафика.  

*Состав*: 
- **tr_gr_id**	- Идентификатор сегмента, первичный ключ
- **bucket_min**	- Минимальная граница сегмента
- **bucket_max**	- Максимальная граница сегмента
- **title**	- Название сегмента

### Таблица **lifetime_segment**  
Данные о выделяемых сегментах по количеству месяцев «жизни» клиента — лайфтайму.  

*Состав*: 
- **lt_gr_id**	- Идентификатор сегмента, первичный ключ
- **bucket_min**	- Минимальная граница сегмента
- **bucket_max**	- Максимальная граница сегмента
- **title**	- Название сегмента

## План работы  
- Определить какие поля из таблиц понадобятся в работе
- Подключиться к базе
- Подготовить SQL-запрос и выгрузить необходимые данные
- Импортировать в Tableau получившуюся выгрузку
- Рассчитать NPS по формуле: % «сторонников» - % «критиков»
- Подготовить графики и дашборды, необходимые для ответа на вопросы исследования
- Сделать выводы
- Подготовить и опубликовать презентацию

## Определение необходимых полей

Нам понадобятся следущие поля:  

- **user_id**	- Идентификатор клиента
- **lt_day**	- Количество дней «жизни» клиента
- **is_new**	- Поле хранит информацию о том, является ли клиент новым
- **age**	- Возраст
- **gender_segment**	- Пол (для удобства работы с полем мы преобразуем значения в текстовый вид)
- **os_name**	- Тип операционной системы
- **cpe_type_name**	- Тип устройства
- **country**	- Страна проживания
- **city**	- Город проживания
- **age_segment**	- Возрастной сегмент
- **traffic_segment**	- Сегмент по объёму потребляемого трафика
- **lifetime_segment**	- Сегмент по количеству дней «жизни»
- **nps_score**	- Оценка клиента в NPS-опросе
- **nps_group**	- Поле хранит информацию о том, к какой группе относится оценка клиента в опросе


Для удобства составления SQL-запроса набросаем схему базы данных, обозначим необходимые нам поля и ключи.  

![](//i.ibb.co/dccVgWC/image.jpg)

## Подключение к базе

In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

## Выгрузка

Согласно описанию данных поле gender_segment содержит значения в виде 0 и 1 (где 0 - мужчина, а 1 - женщина).  
Для удобства мы хотим изменить цифровое значение поля на текстовое, но для начала мы удостоверимся, что в базе отсутствуют пропущенные значения в этом поле.

In [17]:
query1 = """
    SELECT 
           COUNT(*)
    FROM user
    WHERE gender_segment IS NULL
"""

In [18]:
df1 = pd.read_sql(query1, engine)
df1

,COUNT(*)
0,1301


У нас получилось 1301 поле, где не указан пол. Такие поля мы заполним значениями с текстом "Не определен".

In [20]:
query2 = """
    SELECT  user_id,
            lt_day,
            CASE
                WHEN lt_day <= 365 THEN 'True'
                ELSE 'False'
            END as is_new,
            age,
            CASE
                WHEN gender_segment = '1' THEN 'женщина'
                WHEN gender_segment = '0' THEN 'мужчина'
                ELSE 'Не определен'
            END as gender_segment,
            os_name,
            cpe_type_name,
            Country,
            City,
            SUBSTRING (a.title, 4) as age_segment,
            SUBSTRING (t.title, 4) as traffic_segment,
            SUBSTRING (lf.title, 4) as lifetime_segment,
            nps_score,
            CASE
                WHEN nps_score >= 9 THEN 'сторонники'
                WHEN nps_score = 7 OR nps_score = 8 THEN 'нейтралы'
                ELSE 'критики'
            END as nps_group
    FROM user as u
    JOIN location as loc ON u.location_id = loc.location_id
    JOIN age_segment as a ON u.age_gr_id = a.age_gr_id
    JOIN traffic_segment as t ON u.tr_gr_id = t.tr_gr_id
    JOIN lifetime_segment as lf ON u.lt_gr_id = lf.lt_gr_id;
"""

In [22]:
df2 = pd.read_sql(query2, engine)
df2.sample(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
173594,IYNW4J,819,False,65.0,мужчина,ANDROID,SMARTPHONE,Россия,Москва,66 +,25-30,25-36,10,сторонники
263527,NMOJTB,821,False,42.0,мужчина,ANDROID,SMARTPHONE,Россия,Ижевск,35-44,1-5,25-36,10,сторонники
62838,D8IQ2F,2885,False,52.0,мужчина,ANDROID,SMARTPHONE,Россия,Тюмень,45-54,40-45,36+,9,сторонники
311577,Q4D6Z7,5048,False,13.0,мужчина,ANDROID,SMARTPHONE,Россия,Казань,до 16,55-60,36+,10,сторонники
191688,JWN84S,5531,False,56.0,женщина,ANDROID,SMARTPHONE,Россия,НижнийНовгород,55-64,1-5,36+,1,критики
378383,TKSBZP,556,False,21.0,женщина,IOS,SMARTPHONE,Россия,Ижевск,16-24,50-55,13-24,10,сторонники
293108,P5T1FL,955,False,63.0,мужчина,ANDROID,SMARTPHONE,Россия,НижнийТагил,55-64,55-60,25-36,8,нейтралы
35978,BV05ER,2813,False,37.0,мужчина,ANDROID,SMARTPHONE,Россия,Владимир,35-44,0.1-1,36+,10,сторонники
121850,G9ZN6P,2882,False,35.0,мужчина,ANDROID,SMARTPHONE,Россия,Екатеринбург,35-44,50-55,36+,10,сторонники
152609,HVJQA2,586,False,27.0,женщина,ANDROID,SMARTPHONE,Россия,Новосибирск,25-34,15-20,13-24,10,сторонники


In [23]:
df2.to_csv('telecomm_csi_tableau.csv', index=False)

In [24]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502493 entries, 0 to 502492
Data columns (total 14 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   user_id           502493 non-null  object 
 1   lt_day            502493 non-null  int64  
 2   is_new            502493 non-null  object 
 3   age               501939 non-null  float64
 4   gender_segment    502493 non-null  object 
 5   os_name           502493 non-null  object 
 6   cpe_type_name     502493 non-null  object 
 7   country           502493 non-null  object 
 8   city              502493 non-null  object 
 9   age_segment       502493 non-null  object 
 10  traffic_segment   502493 non-null  object 
 11  lifetime_segment  502493 non-null  object 
 12  nps_score         502493 non-null  int64  
 13  nps_group         502493 non-null  object 
dtypes: float64(1), int64(2), object(11)
memory usage: 53.7+ MB


## Ссылка на презентацию в Tableau

https://public.tableau.com/views/_NPS___v2/sheet10?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link